In [1]:
import pandas as pd

In [4]:
import os
import pandas as pd

def generate_data_csv(csv_paths, output_path='data.csv'):
    """
    Create a unified CSV from multiple snr_results.csv files with specific metadata fields.

    Args:
        csv_paths (list): List of paths to snr_result CSV files.
        output_path (str): Output path for the merged data.csv file.
    """
    all_data = []

    for path in csv_paths:
        try:
            df = pd.read_csv(path)

            for _, row in df.iterrows():
                filename = row['filename']
                snr = row['snr_db']

                bonafide_status = 'Spoof' if 'mar_' in filename else 'Bonafide'

                entry = {
                    'BONAFIDE AUDIO FILE': filename,
                    'PLAYBACK DEVICE': '',
                    'RECORDING DEVICE': '',
                    'DISTANCE': '',
                    'ANGULAR ORIENTATION': '',
                    'ROOM ENVIRONMENT': '',
                    'GENDER': '',
                    'SNR': snr
                }

                all_data.append(entry)

        except Exception as e:
            print(f"Failed to process {path}: {e}")

    final_df = pd.DataFrame(all_data)
    final_df.to_csv(output_path, index=False)
    print(f"✅ Merged CSV saved to: {output_path}")

# 🔁 Example Usage
# generate_data_csv([
#     '/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/2M snr_results.csv',
#     '/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/1M snr_results.csv',
#     '/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/5M snr_results.csv',
#     '/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/50CM snr_results.csv'
# ])

In [5]:
generate_data_csv([
    '/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/2M snr_results.csv',
    '/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/1M snr_results.csv',
    '/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/5M snr_results.csv',
    '/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/50CM snr_results.csv'
])

✅ Merged CSV saved to: data.csv


In [1]:
! pip install transformers torchaudio librosa pesq soundfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 2.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 2.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 1.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.1/432.1 kB 2.2 MB/s eta 0:00:0000:0100:01
  Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl (5.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.9/285.9 kB 1.9 MB/s eta 0:00:00a 0:00:01
     ━

In [3]:
! pip install pandas 


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 2.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 1.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
import os
import librosa
import pandas as pd
from pesq import pesq
from transformers import pipeline
import soundfile as sf

# Load Whisper model
whisper_pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small")

def transcribe(audio_path):
    try:
        return whisper_pipe(audio_path)['text']
    except Exception as e:
        print(f"❌ Whisper failed on {audio_path}: {e}")
        return "ERROR"

def compute_pesq(ref_path, deg_path, sample_rate=16000):
    try:
        ref, sr1 = librosa.load(ref_path, sr=sample_rate)
        deg, sr2 = librosa.load(deg_path, sr=sample_rate)

        min_len = min(len(ref), len(deg))
        ref, deg = ref[:min_len], deg[:min_len]

        pesq_val = pesq(sample_rate, ref, deg, 'wb')  # 'wb' = wideband
        return pesq_val
    except Exception as e:
        print(f"❌ Error comparing {ref_path} and {deg_path}: {e}")
        return None

def process_folder_by_sequence(ref_folder, deg_folder, output_csv="pesq_results.csv"):
    ref_files = sorted([f for f in os.listdir(ref_folder) if f.endswith('.wav')])
    deg_files = sorted([f for f in os.listdir(deg_folder) if not f.startswith('.')])  # exclude hidden files

    results = []

    for i in range(min(len(ref_files), len(deg_files))):
        ref_path = os.path.join(ref_folder, ref_files[i])
        deg_path = os.path.join(deg_folder, deg_files[i])

        print(f"🔍 Processing:\nRef: {ref_files[i]}\nDeg: {deg_files[i]}")

        pesq_score = compute_pesq(ref_path, deg_path)
        ref_text = transcribe(ref_path)
        deg_text = transcribe(deg_path)

        results.append({
            "Ref File": ref_files[i],
            "Deg File": deg_files[i],
            "PESQ": pesq_score,
            "Ref Transcript": ref_text,
            "Deg Transcript": deg_text
        })

    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"✅ PESQ results saved to {output_csv}")


Device set to use mps:0


In [8]:
process_folder("/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/real", "/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/spoofed")
process_folder_by_sequence(
    "/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/real",
    "/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/spoofed",
    output_csv="pesq_results.csv"
)

Missing degraded file for 10.wav
Missing degraded file for 9.wav
Missing degraded file for 8.wav
Missing degraded file for 6.wav
Missing degraded file for 7.wav
Missing degraded file for 5.wav
Missing degraded file for 4.wav
Missing degraded file for 1.wav
Missing degraded file for 3.wav
Missing degraded file for 2.wav
✅ PESQ results saved to pesq_results.csv
🔍 Processing:
Ref: 1.wav
Deg: 1.m4a


/var/folders/mz/_mgl45y53018fg9xtlkwt0_m0000gn/T/ipykernel_69099/3699764797.py:21: UserWarning: PySoundFile failed. Trying audioread instead.
  deg, sr2 = librosa.load(deg_path, sr=sample_rate)
/Users/rohitlabade/.pyenv/versions/3.9.18/lib/python3.9/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


KeyboardInterrupt: 

In [1]:
import subprocess
from pathlib import Path

def convert_m4a_folder_to_wav(folder_path: str):
    """
    Convert all .m4a files in folder_path to .wav (mono, 16 kHz)
    Output files are saved in the same folder with the same base name.
    """
    folder = Path(folder_path)

    if not folder.is_dir():
        raise NotADirectoryError(f"{folder} is not a valid folder")

    m4a_files = list(folder.glob("*.m4a"))

    if not m4a_files:
        print("No .m4a files found in the folder.")
        return

    for m4a_file in m4a_files:
        wav_file = m4a_file.with_suffix(".wav")
        cmd = [
            "ffmpeg",
            "-y",              # overwrite output if exists
            "-i", str(m4a_file),
            "-ac", "1",        # mono
            "-ar", "16000",    # 16 kHz
            str(wav_file),
        ]

        print(f"Converting: {m4a_file.name} -> {wav_file.name}")
        subprocess.run(cmd, check=True)

    print("Conversion finished for all .m4a files.")

In [4]:
# Example: if your files are in /Users/rohit/Music/m4a_files
convert_m4a_folder_to_wav("/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/RUJUTA/1m")

Converting: mar_0092_1m.m4a -> mar_0092_1m.wav
Converting: mar_0080_1m.m4a -> mar_0080_1m.wav


ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.4.4.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --

Converting: mar_0049_1m.m4a -> mar_0049_1m.wav
Converting: mar_0034_1m.m4a -> mar_0034_1m.wav
Converting: mar_0057_1m.m4a -> mar_0057_1m.wav
Converting: mar_0026_1m.m4a -> mar_0026_1m.wav


ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.4.4.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --

Converting: mar_0045_1m.m4a -> mar_0045_1m.wav
Converting: mar_0038_1m.m4a -> mar_0038_1m.wav
Converting: mar_0010_1m.m4a -> mar_0010_1m.wav
Converting: mar_0073_1m.m4a -> mar_0073_1m.wav


...

ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.4.4.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --

Converting: mar_0068_1m.m4a -> mar_0068_1m.wav
Converting: mar_0085_1m.m4a -> mar_0085_1m.wav
Converting: mar_0089_1m.m4a -> mar_0089_1m.wav
Converting: mar_0097_1m.m4a -> mar_0097_1m.wav


ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.4.4.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --

Conversion finished for all .m4a files.


[out#0/wav @ 0xc5b0046c0] video:0KiB audio:198KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.038452%
size=     198KiB time=00:00:06.33 bitrate= 256.1kbits/s speed= 769x elapsed=0:00:00.00    


In [5]:
import subprocess
from pathlib import Path

def convert_m4a_to_wav(input_folder: str, output_folder: str):
    """
    Convert all .m4a files in input_folder to .wav (mono, 16 kHz)
    Save the converted files in output_folder with the same base filename.
    """
    input_path = Path(input_folder)
    output_path = Path(output_folder)

    if not input_path.is_dir():
        raise NotADirectoryError(f"{input_path} is not a valid directory")

    # Create output folder if it does not exist
    output_path.mkdir(parents=True, exist_ok=True)

    m4a_files = list(input_path.glob("*.m4a"))

    if not m4a_files:
        print("No .m4a files found in the input folder.")
        return

    for m4a_file in m4a_files:
        wav_file = output_path / (m4a_file.stem + ".wav")
        
        cmd = [
            "ffmpeg",
            "-y",                 # overwrite if exists
            "-i", str(m4a_file),  # input file
            "-ac", "1",           # mono
            "-ar", "16000",       # 16 kHz
            str(wav_file)         # output file
        ]

        print(f"Converting: {m4a_file.name} -> {wav_file.name}")
        subprocess.run(cmd, check=True)

    print("All files converted successfully!")

In [8]:
convert_m4a_to_wav(
    input_folder="/Users/rohitlabade/Desktop/SAMSUNG-LID-MARATHI/RUJUTA/2m",
    output_folder="/Users/rohitlabade/Desktop/LCNN/data/spoof"
)

Converting: mar_0085_2m.m4a -> mar_0085_2m.wav
Converting: mar_0089_2m.m4a -> mar_0089_2m.wav
Converting: mar_0097_2m.m4a -> mar_0097_2m.wav


ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.4.4.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --

Converting: mar_0040_2m.m4a -> mar_0040_2m.wav
Converting: mar_0023_2m.m4a -> mar_0023_2m.wav
Converting: mar_0052_2m.m4a -> mar_0052_2m.wav
Converting: mar_0031_2m.m4a -> mar_0031_2m.wav


ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.4.4.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --

Converting: mar_0064_2m.m4a -> mar_0064_2m.wav
Converting: mar_0019_2m.m4a -> mar_0019_2m.wav
Converting: mar_0007_2m.m4a -> mar_0007_2m.wav
Converting: mar_0076_2m.m4a -> mar_0076_2m.wav


...

ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.4.4.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --

Converting: mar_0045_2m.m4a -> mar_0045_2m.wav
Converting: mar_0038_2m.m4a -> mar_0038_2m.wav
Converting: mar_0010_2m.m4a -> mar_0010_2m.wav
Converting: mar_0073_2m.m4a -> mar_0073_2m.wav


ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.4.4.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --

Converting: mar_0002_2m.m4a -> mar_0002_2m.wav
Converting: mar_0061_2m.m4a -> mar_0061_2m.wav
Converting: mar_0092_2m.m4a -> mar_0092_2m.wav
Converting: mar_0080_2m.m4a -> mar_0080_2m.wav
All files converted successfully!


ffmpeg version 8.0.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.4.4.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --